# Getting started: real-time MPC


Now that you are familiar with the basic MPC setups in **do-mpc**, let's have a look at what you might do next to take your 
design one step closer to a real application.

## Intro
In this Jupyter Notebook we will have a look at setting up and running an example of a **do-mpc** configuration in a real-time environment, which mimics closely the reality encountered in the real world, be it in a lab setup or an industriyl application. The simulator will act as a stand-alone plant which, once started, runs continuously in the background. The MPC controller must have a means of communicating to the plant and sending it optimal inputs at certain time intervals. Additionally, the state of a real plant can rarely, if at all, be fully measured. Therefore a state estimator is typically used and this, too, will require to access plant measurements and provide the estimated state vector to the controller with a certain frequency.

The previous paragraph briefly describes the real-time scenario that we will be implementing. One way of realizing it is to use a distributed, asynchronous and parallel software implementation. The communication between the MPC modules can be implemented using the classical client-server paradigm of software solutions. To be as close as possible to real, industrial applications, we have opted in **do-mpc** for an OPC UA solution which fulfills all requirements and has the benefit of being very relatable for practitioners from the process systems industry and not only. You can read more about FreeOPCUA at their [webpage](https://python-opcua.readthedocs.io/en/latest/index.html). 

The good news is that all of this is very easy to do in Python. All you need to do is to go to this repository and read the instructions for installing this free Python OPCUA library. While you're there, go ahead and also get their sample GUI Client, which will be very useful later on for inspecting the workings of your real-time solution. That's it! 

Spoiler: installing the requirements is best done through *pip*


In [ ]:
pip install freeopcua, opcua-client

## The real-time modules

The following diagram illustrates what you've just read above. It shows the main MPC modules and the way data is exchanged between them via the OPCUA server, in the middle. It is important to mention that each module has its own OPCUA client, embedded away in its core. The data exchange is managed behind the curtains by **do-mpc**, such that each module reads onyl teh relevant data from the server and, conversly, only writes the meaningful stuff back on. For example the simulator only needs to read the latest available inputs. It also only writes the current measured outputs and, for debugging purposes, certain model parameters. 

![rtenvironmentchematic](rt_environment.png)

### The do-mpc Server
The first thing you want to do is to set up your Server and, maybe, a primary client which will allow you to access basic MPC data and other usefull things like flags and module switches. Don't worry, we'll go into more detail about them later.

In [ ]:
# Defining the settings for the OPCUA server
server_opts = {"_model":model,                   # must be the model used to define the controller
               "_name":"Poly Reactor OPCUA",     # give the server whatever name you prefer
               "_address":"opc.tcp://localhost:4840/freeopcua/server/",  # does not need changing
               "_port": 4840,                    # does not need changing
               "_server_type": "with_estimator", # to use with either SFB or EKF estimators or select "basic" for no estimates
               "_store_params": True,            # should always be set to True to spare yourself the headaches
               "_store_predictions": False,      # set to True only if you plan to do fancy things with the predictions
               "_with_db": True}                 # set to True if you plan to use the SQL database during/after the runtime 

# Create your OPUA server
opc_server = Server(server_opts)

The `server_opts` include all the options that are implemented so far. Please note that all except the last three are required in order to create a server. Now that the server is created, we need to start it. On attempting to start the server, the procedure will fail if 1) the instantiation of the server had previously failed or 2) the service is already running. This is by default implmented in the FreeOPCUA routines and we must take care of it here too. 

In [ ]:
# The server can only be started if it hasn't been already created
if opc_server.created == True and opc_server.running == False: opc_server.start()

### Understanding the data structure and the namespace 

By default, **do-mpc** sets up a working data configuration of the server, depending on the settings you provide. The configuration, called a namespace, contains two parts: the functional name of a variable (used to address that variable) and the visible name that appears upon inspecting the server. In the following we describe the default structure. We will see later how the names and structure can be changed.

In [ ]:
"""
"Server Name" ======= The name you define in `server_opts`
 |--> "Plant Data"
 |     |-> "States.X"          = [0..n_x..0]  ======= Differential states are automatically stored.
 |     |-> "States.Z"          = [0..n_z..0]  ======= Algebraic states, if present, are stored.
 |     |-> "Inputs"            = [0..n_u..0]  ======= Actual plant inputs.Not necessaryle identical to the controller outputs.
 |     |-> "Measurements"      = [0..n_y..0]  ======= Measured outputs of the plant
 |    {|-> "Parameters"        = [0..n_p..0]} ======= The (uncertain) parameter set contains all parameters that can be stored.
 |
 |--> "Controller Data"
 |     |-> "InitialState"      = [0....n_x.....0] 
 |     |-> "OptimalOutputs"    = [0....n_u.....0] 
 |    {|-> "StatePredictions"  = [0..n_x_pred..0]} == The future state trajectory, as computed at the `current` time step.
 |    {|-> "OutputPredictions" = [0..n_u_pred..0]} == The future optimal control inputs, as computed at the `current` time step.
 |
{|--> "Estimator Data" 
 |     |-> "Estimates.X"       = [0..n_x..0]  ======= The diffenertial states. Are by default estimated.
 |     |-> "Estimates.Z"       = [0..n_z..0]  ======= The algebraic states. Not yet implemented.
 |    {|-> "Estimates.P"       = [0..n_p..0]} ======= Parameter estimates. Optional.
 |}
 |
 |--> "SupervisionData" 
       |-> Flags               = [0,0,0,0,0]  ======= Signal abnormal operation in one or all of the modules        
       |-> Switches            = [0,0,0,0,0]  ======= Can be used to `manually` switch ON/OFF individual modules
"""

In [ ]:
"""
The structure of the flags vector:

flag = 0 - everything okay
     = 1 - error (see printout error message at the console)
|-------------------------------------------------------------------------------
|   Optimizer   |   Simulator   |   Estimator   |   Monitoring   |   UserDef    |
|-------------------------------------------------------------------------------
|      1/0      |      1/0      |      1/0      |      1/0       |     1/0      |  
--------------------------------------------------------------------------------

The structure of the switches vector:

switch = 0 - the module should not be running/writing to the server (global =0 - force all modules to stop)
       = 1 - the module is running/writing data to the server (global = 1 - force all to work)
|-------------------------------------------------------------------------------
|   Optimizer   |   Simulator   |   Estimator   |   Monitoring   |    Global    |
|-------------------------------------------------------------------------------
|      1/0      |      1/0      |      1/0      |      1/0       |      1/0     |  
--------------------------------------------------------------------------------
"""

In [ ]:
""" 
The user defined server namespace to be implemented on the OPCUA server. 
Contains pairs of the form (elementary MPC variable - readable user name) 
"""
self.namespace = {
  'PlantData'      :{'x':"States.X",'z':"States.Z",'u':"Inputs",'y':"Measurements",'p':"Parameters"},
  'ControllerData' :{'x_init':"InitialState",'u_opt':"OptimalOutputs",'x_pred': "PredictedStates", 'u_pred': "PredictedOutputs"},
  'EstimatorData'  :{'xhat':"Estimates.X",'zhat':"Estimates.Z",'phat':"Estimates.P"},
  'SupervisionData':{'flags':"Flags",'switches':"Switches"}
  }

# This is how you can change entries in the namespace, if you want to rename the state vector...
opc_server.namespace['PlantData']['x'] = "Plant states"
# ... or the optimal input vector, etc. 
opc_server.namespace['ControllerData']['u_opt'] = "Latest inputs"
#...
# Alternatively, you could define an entirely new namespace, making sure it contains all elements!!
opc_server.namespace = your_new_namespace

By manually changing the entries in the namespace you change only the visible tags that appear when inspecting the data structure with a visual OPCUA client, like the one offered by FreeOPCUA. The visible names do not affect the functionality of **do-mpc**.

Also important: one can only make changes before starting the server. Afterwars, the namespace strcture remains fixed.

### The Client

In [ ]:
# Defining the settings for the OPCUA clients, which MUST match the server
client_opts = {"_address":"opc.tcp://localhost:4840/freeopcua/server/", # the basic implementation, can remain as is
               "_port": 4840,                              # should remain as is
               "_client_type": "controller",               # simulator, estimator, user
               "_namespace": opc_server.namespace}         # must match the server, therefore simply copy the namespace 

In [ ]:
# The user is an object that lets the main thread access the OPCUA server for status and flag checks
client_opts['_client_type'] = 'ManualUser'
user = Client(client_opts)
#Connect to the server
user.connect()

# Example 1: read all the operation flags
my_flags   = user.checkFlags()
# or read just one of the 5 flags, at position 0
one_flag   = user.checkFlags(pos = 0)
# Example 2: switch on/off the MPC modules
update_res = user.updateSwitches(switchVal=[1,1,1])

In order to prevent Python errors and further headaches, you'd best be careful with starting and stopping the OPCUA modules. The server, for example, should only be started once. And when you're done with it, you should stop it. For the clients, you should disconnect them before stopping them. If you want to be extra safe, go ahead and also delete the underlying Python objects.   

In [ ]:
# Clients should be disconnected first
user.disconnect()
user.stop()
del(user)

# Stopps the server and remove the 'garbage' object
opc_server.stop()
del(opc_server)


## Example: the polymerization semi-batch reactor

We will return to the server and clients later on. Now let's move onto the actual example. We will implement a real-time version of the Industrial Polymerization Reactor, which you can find in under `examples\industrial_poly_realtime`. Here is the sketch of the process, to start our discussion. 

![reactorsketch](reactor_sketch.png)

The reactor consists of a steel reactor, the main part where the polymerization takes place. Since this reaction is highly exothermic, the process needs to be cooled. This is achieved via two cooling systems: a cooling jacket (J) and an external heat exchanger (EHE). Both of these systems are equipped with additional temperature controllers that ensure that the respective temperature in each of the circuits tracks a desired reference temperature. 

The process is controlled via three manipulated variables: the two reference cooling temperatures meantioned above and the monomer feed rate A. A dynamic model of the system and NMPC control have been published in this paper: [Lucia et al. (2014)](https://cdn.syscop.de/publications/Lucia2014.pdf)

Here is a summary of the model implemented in `examples\industrial_poly_realtime\template_model.py`. The parameters can be found in the paper or in the Python code.

\begin{align}
\dot{m}_{\text{acc}} & = \dot{m}_{\text{F}} \\
\dot{m}_{\text{W}} &= \dot{m}_{\text{F}} \omega_{\text{W,F}} \\
\dot{m}_{\text{A}} &= \dot{m}_{\text{F}} \omega_{\text{A,F}}-k_{\text{R1}}\, m_{\text{A,R}}-k_{\text{R2}}\,
m_{\text{AWT}}\, m_{\text{A}}/m_{\text{ges}} , \\
\dot{m}_{\text{P}} &=  k_{\text{R1}}  m_{\text{A,R}}+p_{1} k_{\text{R2}} m_{\text{AWT}} m_{\text{A}}/ m_{\text{ges}}, \\
\dot{T}_{S} & =  1/(c_{\text{p,S}} m_{\text{S}}) k_{\text{K}} A (T_{\text{R}}-T_{\text{S}})-k_{\text{K}} A (T_{\text{S}}-T_{\text{M}}), \\
\dot{T}_{\text{R}} &= 1/(c_{\text{p,R}} m_{\text{ges}})[\dot{m}_{\text{F}}
c_{\text{p,F}} ( T_{\text{F}}-T_{\text{R}}) +\Delta H_{\text{R}} k_{\text{R1}} m_{\text{A,R}}-k_{\text{K}} A (T_{\text{R}}-T_{\text{S}})-\dot{m}_{\text{AWT}} c_{\text{p,R}}(T_{\text{R}}-T_{\text{EK}})],\\
\dot{T}_{\text{M}} &=  1/(c_{\text{p,W}}m_{\text{M,KW}})[\dot{m}_{\text{M,KW}},c_{\text{p,W}}\left(T_{\text{M}}^{\text{IN}}-T_{\text{M}}\right)+k_{\text{K}} A \left(T_{\text{S}}-T_{\text{M}}\right)],\\
\dot{T}_{\text{EK}} &=  1/(c_{\text{p,R}} m_{\text{AWT}})[\dot{m}_{\text{AWT}}
c_{\text{p,W}}\left(T_{\text{R}}-T_{\text{EK}}\right)\alpha\left(T_{\text{EK}}-T_{\text{AWT}}\right)  +
k_{\text{R2}}\, m_{\text{A}}\, m_{\text{AWT}}\Delta
H_{\text{R}}/m_{\text{ges}}], \\
\dot{T}_{\text{AWT}} &=  [\dot{m}_{\text{AWT,KW}}
c_{\text{p,W}} (T_{\text{AWT}}^{\text{IN}}-T_{\text{AWT}})-\alpha\left(T_{\text{AWT}}-T_{\text{EK}}\right)]/(c_{\text
{p,W}} m_{\text{AWT,KW}}),\\
\text{Where:} & \\
U & =  m_{\text{P}}/(m_{\text{A}}+m_{\text{P}}), \\
m_{\text{ges}} & =  \  m_{\text{W}}+m_{\text{A}}+m_{\text{P}}, \\
k_{\text{R1}}  & =  \  k_{0} e^{\frac{-E_{a}}{R
(T_{\text{R}}+273.15)}}\left(k_{\text{U1}}\left(1-U\right)+k_{\text{U2}}
U\right), \\
k_{\text{R2}}  & =  \  k_{0} e^{\frac{-E_{a}}{R
(T_{\text{EK}}+273.15)}}\left(k_{\text{U1}}\left(1-U\right)+k_{\text{U2}
} U\right), \\
k_{\text{K}}  & =  (m_{\text{W}}\,k_{\text{WS}}+m_{\text{A}}\,
k_{\text{AS}}+m_{\text{P}}\,k_{\text{PS}})/m_{\text{ges}},\\
m_{\text{A,R}} & =  m_\text{A}-m_\text{A}
m_{\text{AWT}}/m_{\text{ges}}\\
\text{Additionally:}& \\
\dot{T}_{adiab} &=  \frac{\Delta H_R}{m_{ges} c_{p,R}} \dot{m}_A - (\dot{m}_W+\dot{m}_A+\dot{m}_P)\frac{m_A \Delta H_R}{m^2_{ges} c_{p,R}} +\dot{T}_{\text{R}}
\end{align}

The last equation is added in order to account for the safety criterium that states that the adiabatic temperature in teh reactor must remain below a certain boundary, more precisely: $T_{adiab} \le 109 °C$

## Model template

The following code sets the *do-mpc* model to the above equations: 

In [ ]:
# algebraic equations
U_m    = m_P / (m_A + m_P)
m_ges  = m_W + m_A + m_P
k_R1   = k_0 * exp(- E_a/(R*T_R)) * ((k_U1 * (1 - U_m)) + (k_U2 * U_m))
k_R2   = k_0 * exp(- E_a/(R*T_EK))* ((k_U1 * (1 - U_m)) + (k_U2 * U_m))
k_K    = ((m_W / m_ges) * k_WS) + ((m_A/m_ges) * k_AS) + ((m_P/m_ges) * k_PS)

# Differential equations
dot_m_W = m_dot_f * w_WF
model.set_rhs('m_W', dot_m_W)
dot_m_A = (m_dot_f * w_AF) - (k_R1 * (m_A-((m_A*m_AWT)/(m_W+m_A+m_P)))) - (p_1 * k_R2 * (m_A/m_ges) * m_AWT)
model.set_rhs('m_A', dot_m_A)
dot_m_P = (k_R1 * (m_A-((m_A*m_AWT)/(m_W+m_A+m_P)))) + (p_1 * k_R2 * (m_A/m_ges) * m_AWT)
model.set_rhs('m_P', dot_m_P)

dot_T_R = 1./(c_pR * m_ges)   * ((m_dot_f * c_pF * (T_F - T_R)) - (k_K *A_tank* (T_R - T_S)) - (fm_AWT * c_pR * (T_R - T_EK)) + (delH_R * k_R1 * (m_A-((m_A*m_AWT)/(m_W+m_A+m_P)))))
model.set_rhs('T_R', dot_T_R)
model.set_rhs('T_S', 1./(c_pS * m_S)     * ((k_K *A_tank* (T_R - T_S)) - (k_K *A_tank* (T_S - Tout_M))))
model.set_rhs('Tout_M', 1./(c_pW * m_M_KW)  * ((fm_M_KW * c_pW * (T_in_M - Tout_M)) + (k_K *A_tank* (T_S - Tout_M))))
model.set_rhs('T_EK', 1./(c_pR * m_AWT)   * ((fm_AWT * c_pR * (T_R - T_EK)) - (alfa * (T_EK - Tout_AWT)) + (p_1 * k_R2 * (m_A/m_ges) * m_AWT * delH_R)))
model.set_rhs('Tout_AWT', 1./(c_pW * m_AWT_KW)* ((fm_AWT_KW * c_pW * (T_in_EK - Tout_AWT)) - (alfa * (Tout_AWT - T_EK))))
model.set_rhs('accum_monom', m_dot_f)
model.set_rhs('T_adiab', delH_R/(m_ges*c_pR)*dot_m_A-(dot_m_A+dot_m_W+dot_m_P)*(m_A*delH_R/(m_ges*m_ges*c_pR))+dot_T_R)


Next we must also define the output structure to match the reality of the measured outputs:   

In [ ]:
# The output function, in this case a linear dependency
model.set_meas('m_W_meas', m_W)
model.set_meas('T_S_meas', T_S)
model.set_meas('T_R_meas', T_R)
model.set_meas('T_M_meas', Tout_M)
model.set_meas('T_EK_meas', T_EK)
model.set_meas('T_AWT_meas', Tout_AWT)
model.set_meas('accum_monom_meas', accum_monom)

The model has been implemented. Let us have a look at the optimization problem that we want to solve, both in the classical synchronous mode of *do-mpc*, as well as in the real-time mode. 

For this example we will only consider the single-stage NMPC implementation. The optimization problem that is genrated by *do-mpc* reads:

\begin{align}
 \min_{x,u,p} & \sum ^{Np}_{k=0} J_{p,k} \\
 \text{subject to:}&\\
              x_{k+1} & = f(x_k, u_k, p_k) \\
              88 °C & \le T_R \le 92 \\
              T_{adiab} & \le 109 
\end{align}

## Controller template

Let us have a look at the controller implementation and see how the NMPC controller is defined. The defnition is found in the file `examples\industrial_poly_realtime\template_mpc.py` 

In [ ]:
# The controller is typically run less often than the simulator/estimator
opc_opts['_opc_opts']['_client_type'] = "controller"
opc_opts['_opc_opts']['_output_feedback'] = True
opc_opts['_cycle_time'] = 10.0

controller = RealtimeController(model,opc_opts)

The real-time controller is very easy to create. The type must be declared as `controller`, ensuring that the OPCUA client reads and writes data correctly on the server. 
Next, we set the output feedback to `False` to start. We want to make sure that the controller runs okay in a state-feedback setup. Finally, we want to call the controller every 10 seconds, which is a first guess meant to provide the rea-time structure some buffer time and ensure that the optimization can finish within the cycle time of the controller (we mighjt have to adjust this interval later). 

In [ ]:
setup_mpc = {
    'n_horizon': 20,
    'n_robust': 1,
    'open_loop': 0,
    't_step': 10.0/3600.0,
    'state_discretization': 'collocation',
    'store_full_solution': True,
    # Use MA27 linear solver in ipopt for faster calculations:
    #'nlpsol_opts': {'ipopt.linear_solver': 'MA27'}
}

controller.set_param(**setup_mpc)

_x, _u, _z, _tvp, p, _aux,  *_ = controller.model.get_variables()

The NMPC controller is not fully initialized until all the NLP settings have been passed. For further details about the initialization please have a look at our other, more basic example, the `getting_started:MPC` Jupyter Notebook. 

In [ ]:
temp_range = 2.0
#...
controller.bounds['lower','_x','m_P'] = 26.0
controller.bounds['lower','_x','T_R'] = 363.15 - temp_range
# ...
controller.bounds['upper','_x','T_R'] = 363.15 + temp_range
controller.bounds['upper','_x','accum_monom'] = 30000.0
# ...

In [ ]:
# Cost function definition
mterm = - _x['m_P']
lterm = - _x['m_P']

controller.set_objective(mterm=mterm, lterm=lterm)
controller.set_rterm(m_dot_f=0.002, T_in_M=0.004, T_in_EK=0.002)


The cost function here is composed of three kinds of terms: the Lagrange term, the Mayer term and the input penalty term. For the last one of them, the interface allows you to set individual penalty terms for each one of the controll variables.

In [ ]:
# Soft constraints
controller.set_soft_constraints('_x', 'T_adiab', )

## Simulator template

You can use the simulator in order to just simulate the plant overa given time span. However, we generally want to do something more with our model: test estimators and NMPC controllers. Let'S first get the simulator set up and then we will move on to the other modules. 

In [ ]:
# The simulator is the one that typically run the fastest (most often, e.g every second)
opc_opts['_opc_opts']['_client_type'] = "simulator"
opc_opts['_cycle_time'] = 2.0
simulator = RealtimeSimulator(model,opc_opts)


In [ ]:
params_simulator = {
    'integration_tool': 'cvodes',
    'abstol': 1e-10,
    'reltol': 1e-10,
    't_step': 2.0/3600.0
}

simulator.set_param(**params_simulator)

In [ ]:
# Set the real value of the [potentially] uncertain parameters
p_num = simulator.get_p_template()
p_num['delH_R'] = 950.0
p_num['k_0'] = 7.0

# The parameters are returned by a user-defined function, here a dummy implementation
def p_fun(t_now):
    # that returns the same values always
    return p_num
simulator.set_p_fun(p_fun)
# And the last step ...
simulator.setup()

## Estimator template

The next section will give you a short overview on setting up a simple estimator. As of now, **do-mpc** offers you a choice of two estimation methods: Extended Kalman Filter (EKF) and the Moving Horizon Estimator (MHE). Add to those the choice of using full State FeedBack (SFB) and these are your options. For more information about the MHE, please read through our other example at [`Getting started: MHE`](mhe_example.ipynb)

In [ ]:
# The estimator is just a delayed state feedback estimator in this case 

opc_opts['_opc_opts']['_client_type'] = "estimator"
opc_opts['_cycle_time'] = 3.0   # execute every 3 seconds
opc_opts['_opc_opts']['_output_feedback'] = False

# Use type 'SFB' for a full state feedback or 'EKF'/'MHE' for an actual estimator
etype = 'SFB'
if etype == 'SFB':
    estimator = RealtimeEstimator('SFB', model, opc_opts)

# And finally set it up
estimator.setup()

The real-time estimator module will be executed every 3 seconds. Again, we start off by disabling the real-time feedback, because we want to first check that the estimator performs well and tune it.

For the real-time implementation, you have a choice of only two real-time estimators: 'SFB' stands for a full-state feedback, which asssumes that you can basically measure all plant states. This is perfect for setting up the controller and testing the rea-time environment. Later on, you can switch to 'EKF', which implements an extended Kalman Filter. For further information about estimators, we recommend reading through our [`Getting started: do-mpc`](getting_started.ipynb) notebook and the CSTR example. 

For completeness, the template includes the following code, that can be used to define the real-time EKF. Note that the matrix C must match the measurement structure defined in the model above, or else the estimator will abort at initialization. 

In [ ]:
 if etype == 'EKF':
    estimator = RealtimeEstimator('EKF', model, opc_opts)

    
    """ The following state and measurement structures are implemented:
        x  = [m_W, m_A, m_P, T_R, T_S, Tout_M, T_EK, T_AWT, accum_monom, T_adiab]
        y  =   |    0    0    |    |      |     |     |          |        0
    """
    # The C matrix must be defined according to the above assumnption, has therefore size (7x10)
    C = np.matrix([[1,0,0,0,0,0,0,0,0,0],
                   [0,0,0,1,0,0,0,0,0,0],
                   [0,0,0,0,1,0,0,0,0,0],
                   [0,0,0,0,0,1,0,0,0,0],
                   [0,0,0,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,0,1,0,0],
                   [0,0,0,0,0,0,0,0,1,0]])  
    
    # Tuning paramaters EKF
    Q = 0.001*np.eye(10)
    R = 0.01*np.eye(7)
    P0 = 100*Q

    setup_ekf = {
    'P0': P0,
    'Q' : Q,
    'R' : R,
    'C' : C,
    't_step': 3.0/3600.0,
    'type': "continuous_discrete",
    'estimate_params': False,
    'output_func':'linear', 
    'noise_level':0.01
    #'output_func':'nonlinear',
    #'H_func': h
    }
       
    estimator.set_param(**setup_ekf) 
    estimator.setup()

## The structure of the main file 

We change our attention to the main *do-mpc* file, where we must first import the files containing the definition of the real-time modules.  

In [ ]:
from template_model import template_model
from template_mpc import template_mpc
from template_simulator import template_simulator
from template_estimator import template_estimator
from template_opcua import template_opcua

from opcmodules import Server, Client
from opcmodules import RealtimeSimulator, RealtimeController, RealtimeEstimator
from opcmodules import RealtimeTrigger

Next we set up all rea-time modules by calling the templates and collecting the resulting MPC modules.

In [ ]:
model = template_model()

opc_server, opc_opts = template_opcua(model)
    
rt_simulator = template_simulator(model,opc_opts)

rt_estimator = template_estimator(model, opc_opts)

rt_controller = template_mpc(model,opc_opts)

Notice that the real-time modules have a different calling syntax, compared to their synchronous counterparts. The options related to the OPCUA server and connection setting, as well as other settings are passed via the `opc_opts` dictionary. Please see the `template_opcua.py` file and the code documentation for further information. 

We must initialize the data structure on the server with the starting point of our simulation. FIrst, the $x_0$ state of the plant, controller and estimator must be defined.

In [ ]:
# Set the initial state of mpc and simulator:
x0 = model._x(0)

delH_R_real = 950.0
c_pR = 5.0

x0['m_W'] = 10000.0
x0['m_A'] = 853.0
x0['m_P'] = 26.5

x0['T_R'] = 90.0 + 273.15
x0['T_S'] = 90.0 + 273.15
x0['Tout_M'] = 90.0 + 273.15
x0['T_EK'] = 35.0 + 273.15
x0['Tout_AWT'] = 35.0 + 273.15
x0['accum_monom'] = 300.0
x0['T_adiab'] = x0['m_A']*delH_R_real/((x0['m_W'] + x0['m_A'] + x0['m_P']) * c_pR) + x0['T_R']

In [ ]:
# Step 1: initilize the simulator part
rt_simulator.set_initial_state(x0, reset_history=True)
rt_simulator.init_server(x0.cat.toarray().tolist())

# Step 2: initialize the estimator part (if present)
rt_estimator.set_initial_state(x0, reset_history=True)
rt_estimator.init_server(x0.cat.toarray().tolist())

# Step 3: only now can the optimizer be initialized, and a first optimization can be executed
time.sleep((rt_simulator.cycle_time+rt_estimator.cycle_time)/2)
rt_controller.set_initial_state(x0, reset_history=True)
rt_controller.init_server(rt_controller._u0.cat.toarray().tolist())

Notice that we are calling the routine `RealtimeSimulator.init_server()` for each of the modules. This has the effect of making the first update on the server, so that once we start the modules, they will read and work with relevant data. This is only done in order to prevent runtime failuers, for example by the optimizer having to start from an all-0 vector, which is physically not meaningful.  

In [ ]:
# Step 4: the cyclical operation can be safely started now
"""
Define triggers for each of the modules and start the parallel/asynchronous operation
"""
trigger_simulator  = RealtimeTrigger(rt_simulator.cycle_time , rt_simulator.asynchronous_step)

trigger_estimator  = RealtimeTrigger(rt_estimator.cycle_time , rt_estimator.asynchronous_step)

trigger_controller = RealtimeTrigger(rt_controller.cycle_time, rt_controller.asynchronous_step)


The real-time execution of the modules must be managed asynchronously and according to the cycle times defined by teh user in the templates. The special tool for doing this is `RealtimeTrigger`, a class that was implemented in **do-mpc** in order to easily manage the execution. This class takes a cycle time (in seconds) and a function (as Python function handle) and executes it at the given frequency. Once the trigger is instantiated, the execution of the thread starts automatically.

The execution of each one of the functions happens independently of the others, while the user still maintains control of the main thread of execution, as we will see later. 

In [ ]:
# Once the main thread reaches this point, all real-time modules will be stopped
trigger_controller.stop()
trigger_simulator.stop()
trigger_estimator.stop()

Meanwhile, you can implement routine checks or message displays on the main thread. For the easiest configuration, we recommend that you let the execution start automatically when you run the main file, i.e. set `opc_opts['user_controlled'] = False`. Alternatively, you can use the `ManualUser` client to start and stop the execution quasi-automatically, as we will show next:

In [ ]:
# Start all modules if the manual mode is enabled and avoid delays
if opc_opts['_user_controlled']: 
    user.updateSwitches(pos = -1, switchVal=[1,1,1])

Now all the modules are started and running as defined. The code below represents the main thread. You will that the main thread is executed every 10 seconds and it performs some very basic message displays and server checks, via our previously defined `ManualUser` interface. 

In [ ]:
"""
The real-time do-mpc will keep running until you manually stop it via the flags (use an OPCUA Client to set the flags).
Alternatively, use the routine below to check when the maximum nr of iterations is reached and stop the real-time modules.
"""
max_iter = 100
manual_stop = False

while rt_controller.iter_count < max_iter and manual_stop == False:
    # The code below is executed on the main thread (e.g the Ipython console you're using to start do-mpc)
    print("Waiting on the main thread...Checking flags...Executing your main code...")
    
    if user.checkFlags(pos=0) == 1:
        print("The controller has failed! Better take backup action ...")
    if user.checkFlags(pos=0) == 0: print("All systems OK @ ", time.strftime('%Y-%m-%d %H:%M %Z', time.localtime()))
    
    # Checking the status of the modules
    switches = user.checkSwitches()
    print("The controller is:", 'ON' if switches[0] else 'OFF')
    print("The simulator  is:", 'ON' if switches[1] else 'OFF')
    print("The estimator  is:", 'ON' if switches[2] else 'OFF')
    
    # Check the 5th flag and stop all modules at once if the user has raised the flag
    # Alternatively, the user can individually stop modules by setting the switch to 0
    if user.checkFlags(pos=4) == 1: 
        user.updateSwitches(pos = -1, switchVal=[0,0,0])
        manual_stop = True
    
    # The main thread sleeps for 10 seconds and repeats
    time.sleep(10)

## Running the code, getting results

In this example, we want to execute 100 NMPC iterations of the polymerization reactor test case. However, if you decide to stop the modules before that, you can always connect to the server and set the switches for any or all of the modules to 0. This will cause the module(s) to stop after finishing their current iteration. 

Now let us look at some results. The next two images will show: 1) 100 iteration of real-time NMPC with full state-feedback vs 2) 100 iterations of the same setup, but executed in simulated time, with the 'classic' **do-mpc** setup.

You should notice that the real-time solution exhibits small oscillations, which become visible after t=0.2 h, when the adiabatic temperature hits the upper soft constraint. This is normal, although counterintuitive at first. After all, you can see that the simulated time solution 1) does not exhibit such a behavior. For the long answer to this, please consider reading our paper.

Here is the short answer: the asynchronous execution introduces implicit time delays, which, especially at the limit of feasibility, become quite important. They can cause the solution of the optimization to oscillate or even crash. This is to be expected, if we think back and realize the optimizer always computes optimal inputs for a past state of the plant. By the time the inputs are updated, say after 10 seconds, the real plant has already evolved past this point, making the inputs suboptimal at the new state of the plant. Further delays can be introduced by the estimator, which interlaces itself in-between the plant and the controller.

Several solutions exist for this problem, and they are explained in the above paper. We have added one final result in 3), where we show that the real-time framework can be modified in order to cope with these inherent time delays. 

## Testing the EKF estimator and output-feedback